In [1]:
%load_ext autoreload
%autoreload 2   

In [2]:
from typing import Any
import re
from enum import Enum

In [3]:
from tools.writer import EnumEntry, enum_file, write_enum_class, AUTO_ENUMS_PATH

In [4]:
from statscan.util.log import configure_logging
from statscan.wds.client import WDS

In [5]:
configure_logging(level='DEBUG')

In [6]:
client = WDS()

In [7]:
codesets: dict = await client.getCodeSets()

2025-08-25 14:00:26,733 :: DEBUG :: httpcore.connection :: _trace.atrace:87 - connect_tcp.started host='www150.statcan.gc.ca' port=443 local_address=None timeout=5.0 socket_options=None
2025-08-25 14:00:26,775 :: DEBUG :: httpcore.connection :: _trace.atrace:87 - connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x3678bf99d90>
2025-08-25 14:00:26,775 :: DEBUG :: httpcore.connection :: _trace.atrace:87 - start_tls.started ssl_context=<ssl.SSLContext object at 0x3679b5b0c90> server_hostname='www150.statcan.gc.ca' timeout=5.0
2025-08-25 14:00:26,836 :: DEBUG :: httpcore.connection :: _trace.atrace:87 - start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x3678b52d490>
2025-08-25 14:00:26,837 :: DEBUG :: httpcore.http11 :: _trace.atrace:87 - send_request_headers.started request=<Request [b'GET']>
2025-08-25 14:00:26,838 :: DEBUG :: httpcore.http11 :: _trace.atrace:87 - send_request_headers.complete
2025-08-25 14:00:26,838 :: DEBUG

In [8]:
o: dict[str, list[dict[str, str]]] = codesets['object']

In [9]:
def get_keys(entry_dict: dict[str, Any]) -> tuple[str, str, set[str]]:
    '''
    get the key-key, value-key, and remaining keys
    '''
    remaining_keys = set()
    key_key = None
    value_key = None

    for k in entry_dict.keys():
        if k.endswith("DescEn"):
            key_key = k
        elif k.endswith("Code"):
            value_key = k
        else:
            remaining_keys.add(k)
    
    if key_key is None:
        for k in entry_dict.keys():
            if k.endswith("En"):
                key_key = k
                remaining_keys.remove(k)
                break
        if key_key is None:
            raise ValueError(f"Could not find key key for enum. Possible keys are {entry_dict.keys()}")
    if value_key is None:
        for k in entry_dict.keys():
            if k.endswith("Id"):
                value_key = k
                remaining_keys.remove(k)
                break
        if value_key is None:
            raise ValueError(f"Could not find value key for enum. Possible keys are {entry_dict.keys()}")

    return key_key, value_key, remaining_keys


In [10]:
def camel_to_snake(camel: str) -> str:
    """
    Convert a camelCase string to snake_case.
    """
    snake = re.sub(r'(?<!^)(?=[A-Z])', '_', camel).lower()
    return snake

In [ ]:
for k, v in o.items():
    entries: list[EnumEntry] = []
    cls_name = k[0].upper() + k[1:]
    print(f'Processing Enum: {cls_name}')
    key_key, value_key, remaining_keys = get_keys(v[0])
    print(f"  Key: {key_key}, Value: {value_key}, Other keys: {remaining_keys}")
    for entry_dict in v:
        try:
            entry_key = entry_dict[key_key]
            entry_value = entry_dict[value_key]
        except KeyError as ke:
            print(f"Error processing entry in {cls_name}: {ke}. Available keys are: {entry_dict.keys()}")
        entry_comment = str({k: v for k, v in entry_dict.items() if k in remaining_keys}) if remaining_keys else None
        if entry_key is None:
            entry_key = 'None'
        entry = EnumEntry(key=entry_key, value=int(entry_value), comment=entry_comment)
        
        entries.append(entry)

    # for e in entries:
    #     try:
    #         print(f"  {e.to_string()}")
    #     except Exception as ex:
    #         raise ValueError(f"Failed to process entry {e}. {o[k]}") from ex

    with enum_file(fp=AUTO_ENUMS_PATH / 'wds' / f'{camel_to_snake(k)}.py', imports={'enum': 'Enum'}, overwrite=True) as f:
        write_enum_class(f, entries=entries, cls_name=cls_name, skip_methods=True)
    # print(f"Key: {key_key}, Value: {value_key}")
    # print(f"{k}: {v}")

2025-08-25 14:07:22,995 :: DEBUG :: tools.writer :: <string>.write_enum_class:12 - Writing class Scalar to file from template None...
2025-08-25 14:07:22,996 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: units is unique
2025-08-25 14:07:22,996 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: tens is unique
2025-08-25 14:07:22,997 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: hundreds is unique
2025-08-25 14:07:22,998 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: thousands is unique
2025-08-25 14:07:22,998 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: tens of thousands is unique
2025-08-25 14:07:22,999 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: hundreds of thousands is unique
2025-08-25 14:07:23,000 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: millions is unique
2025-08-25 14:07:23,001 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: tens of milli

2025-08-25 14:07:23,025 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: data quality: very good is unique
2025-08-25 14:07:23,026 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: data quality: good is unique
2025-08-25 14:07:23,027 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: data quality: acceptable is unique
2025-08-25 14:07:23,027 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: use with caution is unique
2025-08-25 14:07:23,028 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: too unreliable to be published is unique
2025-08-25 14:07:23,029 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: not applicable is unique
2025-08-25 14:07:23,029 :: INFO :: tools.writer :: <string>.write_enum_class:32 - wrote the following enum key names: {'NOT_AVAILABLE_FOR_A_SPECIFIC_REFERENCE_PERIOD', 'USE_WITH_CAUTION', 'NORMAL', 'NOT_APPLICABLE', 'DATA_QUALITY_GOOD', 'DATA_QUALITY_EXCELLENT', 'VALUE_ROUNDED_TO_

Processing Enum: Scalar
  Key: scalarFactorDescEn, Value: scalarFactorCode, Other keys: {'scalarFactorDescFr'}
Processing Enum: Frequency
  Key: frequencyDescEn, Value: frequencyCode, Other keys: {'frequencyDescFr'}
Processing Enum: Symbol
  Key: symbolDescEn, Value: symbolCode, Other keys: {'symbolRepresentationEn', 'symbolRepresentationFr', 'symbolDescFr'}
Processing Enum: Status
  Key: statusDescEn, Value: statusCode, Other keys: {'statusDescFr', 'statusRepresentationFr', 'statusRepresentationEn'}
Processing Enum: Uom
  Key: memberUomEn, Value: memberUomCode, Other keys: {'memberUomFr'}


2025-08-25 14:07:23,195 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Tons per acre is unique
2025-08-25 14:07:23,196 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Troy ounces is unique
2025-08-25 14:07:23,197 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Twenty feet equivalent units is unique
2025-08-25 14:07:23,198 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: US dollars per unit of real GDP is unique
2025-08-25 14:07:23,199 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: United States Dollars is unique
2025-08-25 14:07:23,200 :: WARNING :: tools.writer :: <string>.write_enum_class:27 - Key United States dollars is duplicated in Uom, renaming to United States dollars_1.
2025-08-25 14:07:23,200 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: United States dollars per Canadian dollar is unique
2025-08-25 14:07:23,201 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: 198

Processing Enum: Survey
  Key: surveyEn, Value: surveyCode, Other keys: {'surveyFr'}


2025-08-25 14:07:23,602 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Pension Plans in Canada is unique
2025-08-25 14:07:23,602 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Unemployment Insurance Statistics (Annual) is unique
2025-08-25 14:07:23,603 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Survey of Employment, Payrolls and Hours is unique
2025-08-25 14:07:23,604 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Business Payrolls Survey is unique
2025-08-25 14:07:23,605 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Workplace and Employee Survey is unique
2025-08-25 14:07:23,606 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Survey of Financial Security is unique
2025-08-25 14:07:23,606 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Airport Activity Survey is unique
2025-08-25 14:07:23,607 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Air Passen

Processing Enum: Subject
  Key: subjectEn, Value: subjectCode, Other keys: {'subjectFr'}


2025-08-25 14:07:24,299 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Health/Health care services/Residential care is unique
2025-08-25 14:07:24,299 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Health/Health care services/Home care and caregivers is unique
2025-08-25 14:07:24,300 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Health/Health care services/Other content related to Health care services is unique
2025-08-25 14:07:24,300 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Health/Life expectancy and deaths/Causes of death is unique
2025-08-25 14:07:24,301 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Health/Life expectancy and deaths/Infant mortality and fetal deaths (stillbirths) is unique
2025-08-25 14:07:24,301 :: DEBUG :: tools.writer :: <string>.write_enum_class:29 - key: Health/Life expectancy and deaths/Life expectancy is unique
2025-08-25 14:07:24,302 :: DEBUG :: tools.writer :: <strin

Processing Enum: Classificationtype
  Key: classificationTypeEn, Value: classificationTypeCode, Other keys: {'classificationTypeFr'}
Processing Enum: Securitylevel
  Key: securityLevelDescEn, Value: securityLevelCode, Other keys: {'securityLevelRepresentationFr', 'securityLevelDescFr', 'securityLevelRepresentationEn'}
Processing Enum: Terminated
  Key: codeTextEn, Value: codeId, Other keys: {'codeTextFr', 'displayCodeFr', 'displayCodeEn'}
Processing Enum: Wdsresponsestatus
  Key: codeTextEn, Value: codeId, Other keys: {'codeTextFr'}
